# Introduction à DSPy
Ce notebook va servir d'introduction à DSPy, un package dont le but est d'optimiser les prompts et les poids des llm.
Au lieu de se concentrer sur du prompt engineering comme sur langchain, DSPy propose une approche plus classique pour des Data Scientist :
 - on crée un dataset comprenant des exemples de tâches que l'on souhaite faire au llm (ex: question + réponse attendue),
 - on "entraîne" puis valide notre llm sur des sous-ensembles de notre dataset, comme pour entraîner un modèle classique,
 - on sauvegarde notre llm, que l'on pourra charger plus tard pour executer un tâche.

#### Sur ce, commençons par importer des modules et initialiser quelques variables.

In [16]:
import dspy
from dspy.teleprompt import BootstrapFewShot
from dotenv import load_dotenv

In [17]:
load_dotenv()

turbo = dspy.OpenAI(model='gpt-3.5-turbo')

dspy.settings.configure(lm=turbo)

#### Après avoir configuré le llm par défaut, il suffit de lui envoyer une chaîne de caractères pour simplement intéragir avec lui :

In [3]:
turbo("Quelle est la couleur du cheval Blanc d'Henri IV ?")

["Le cheval Blanc d'Henri IV est blanc."]

#### DSPy possède une première classe essentielle, les Signatures.
Les Signatures DSPy sont un moyen de faire comprendre au llm les inputs, outputs et tâches à accomplir avec une écriture très compacte.

Une Signature est données par la structure "input" -> "output" où le nom des variables d'entrée/sortie doivent avoir du sens car seront utilisés par le llm.
Par exemple :

- Question Answering: "question -> answer"

- Sentiment Classification: "sentence -> sentiment"

- Summarization: "document -> summary"

Essayons par exemple de faire un résumé de texte.

In [4]:
texte_long = """
    Le boson de Higgs ou boson BEH, est une particule élémentaire dont l'existence, postulée indépendamment en juin 1964 par François Englert et Robert Brout, par Peter Higgs, en août, 
    et par Gerald Guralnik, Carl Richard Hagen et Thomas Kibble, permet d'expliquer la brisure de l'interaction unifiée électrofaible (EWSB, pour l'anglais electroweak symmetry breaking)
    en deux interactions par l'intermédiaire du mécanisme de Brout-Englert-Higgs-Hagen-Guralnik-Kibble et d'expliquer ainsi pourquoi certaines particules ont une masse et d'autres 
    n'en ont pas."""

résumateur = dspy.Predict("paragraph -> summary")
résumateur(paragraph=texte_long).summary

"Summary: Le boson de Higgs, également connu sous le nom de boson BEH, est une particule élémentaire dont l'existence a été postulée par plusieurs scientifiques en 1964. Il permet d'expliquer la brisure de l'interaction unifiée électrofaible et pourquoi certaines particules ont une masse tandis que d'autres n'en ont pas."

Il suffit de changer le nom de l'output pour changer la fonction du programme. Faire un "citeur_dinventeur" qui va donner le nom des inventeurs d'une expérience.

In [5]:
citeur_dinventeur = dspy.Predict("paragraph -> inventor_names")
citeur_dinventeur(paragraph=texte_long).inventor_names

'Inventor Names: François Englert, Robert Brout, Peter Higgs, Gerald Guralnik, Carl Richard Hagen, Thomas Kibble'

On remarque qu'on a pas besoin de préciser quel llm la signature va utiliser puisqu'il est définit en haut.

In [6]:
contexte = "Henri IV avait un cheval dénommé Blanc qui était en réalité d'un pelage opposé de blanc..."

qa = dspy.Predict("context, question -> answer")
qa(context=contexte, question="Quelle est la couleur du cheval Blanc d'Henri IV ?").answer

"La couleur du cheval Blanc d'Henri IV est blanc."

Le llm n'a pas compris. Predict est la signature de base mais on peut dire au llm de raisonner en utilisant la signature ChainOfThought.

PS : essayez de ne pas regarder que l'attribut "answer"

In [7]:
qa = dspy.ChainOfThought("context, question -> answer")
qa(context=contexte, question="Quelle est la couleur du cheval Blanc d'Henri IV ?").answer

'The color of the horse Blanc, owned by Henri IV, is black.'

Si l'on veut ajouter des contraintes pour des tâches plus spécifiques, on peut raffiner une signature ou en créer une nous-même. Par exemple, disons qu'on veut une réponse plus courte et en français :

In [8]:
class reponse_concise(dspy.Signature):
    """
    Return the answer to a question using context. Keep the answer short and return it in French.
    """
    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Short answer, less then 5 words. Must be in French.")

qa = dspy.ChainOfThought(reponse_concise)
qa(context=contexte, question="Quelle est la couleur du cheval Blanc d'Henri IV ?").answer

'Noir.'

Maintenant que nous sommes des pros des signatures, nous pouvons passer aux modules. Si une Signature est comme une couche de réseau de neuronnes, les modules sont comme les modèles complets. On peut y mettre plusieurs signatures qui s'occupent de différentes tâches, par exemple si on veut reprendre notre exemple où l'on veut que notre llm réponde juste "noir", on peut le décomposer en un llm qui répond, puis un qui traduit, ce qui donnerait ça :

In [9]:
class qa_en_français(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
        self.translate = dspy.Predict("sentence -> french_translation")
    
    def forward(self, context, question):
        eng_answer = self.generate_answer(context=context, question=question)
        french_answer = self.translate(sentence=eng_answer.answer)
        return french_answer

qa = qa_en_français()
qa(context=contexte, question="Quelle est la couleur du cheval Blanc d'Henri IV ?").french_translation

'Sentence: The color of the horse Blanc, owned by Henri IV, is black.\nFrench Translation: La couleur du cheval Blanc, appartenant à Henri IV, est noire.'

Pas mal mais il renvoie "sentence : .... French Translation :" alors qu'on voudrait juste la réponse.
C'est là qu'intervient toute la magie de DSPy : au lieu de perdre du temps à faire du prompt engenireeing sans trop savoir où on va, il nous suffit de produire 3 exemples du comportement qu'on attendrais et d'entraîner notre module sur ces exemples. Commençons :

In [10]:
ex1 = dspy.Example(context="Double-birds are a new species of flying animals with 4 eyes and 3 wings", question="How many wings does a double-bird have ?", answer="Les double-oiseaux ont 3 ailes.").with_inputs("context", "question")

ex2 = dspy.Example(context="", question="Combien de cm y a-t-il dans 1m", answer="Il y a cent centimètres dans un mètre.").with_inputs("context", "question")

ex3 = dspy.Example(context=texte_long, question="Quel est l'autre nom du boson de Higgs ?", answer="Le Boson de Higgs est aussi appelé Boson BEH").with_inputs("context", "question")

Il nous faut maintenant définir une fonction de coût. Tout est possible tant que ça renvoie un Booléen.
On peut checker que la phrase soit exactement comme notre label, on peut check que certains mot-clés soient dans notre réponse (par ex, checker que "Boson BEH" apparaisse dans notre réponse à l'exemple 3).

Par exemple, si on voulait checker que la réponse soit exactement la même, on pourrait utiliser cette loss :

In [11]:
def exact_match(example, pred, trace=None):
    return example.answer.lower() == pred.french_translation.lower()

Cependant, dans notre cas, on s'en fout si on a pas la même phrase au mot près. On veut juste une réponse valide en français. Et comme c'est un peu dur à expliciter et qu'on a de toute façon pas beaucoup d'exemples, on va utiliser une fonction de coût super pratique : nous-même !

In [12]:
def check_truth(example, pred, trace=None):
    print(example["question"])
    print("\n")
    print(pred.french_translation)
    response = input("Is the answer good ? (y/n): ")
    return response.lower() == "y"

On peut maintenant entrainer notre modèle sur ces exemples :

In [13]:
trainset = [ex1, ex2, ex3]
metric = check_truth
teleprompter = BootstrapFewShot(
    metric=metric,
    max_rounds=3,
)
trained_qa = teleprompter.compile(
    qa, trainset=trainset
)

  0%|          | 0/3 [00:00<?, ?it/s]

How many wings does a double-bird have ?


Sentence: A double-bird has 3 wings.
French Translation: Un oiseau double a 3 ailes.


 33%|███▎      | 1/3 [00:05<00:10,  5.26s/it]

Combien de cm y a-t-il dans 1m


Il y a 100 centimètres dans 1 mètre.


 67%|██████▋   | 2/3 [00:11<00:05,  5.83s/it]

...

Quel est l'autre nom du boson de Higgs ?


L'autre nom du boson de Higgs est le boson BEH.


  0%|          | 0/3 [00:00<?, ?it/s]

How many wings does a double-bird have ?


Sentence: A double-bird has 3 wings.
French Translation: Un oiseau double a 3 ailes.


100%|██████████| 3/3 [00:06<00:00,  2.24s/it]

Bootstrapped 2 full traces after 3 examples in round 2.


In [15]:
trained_qa(context=contexte, question="Quelle est la couleur du cheval Blanc d'Henri IV ?").french_translation

"La couleur du cheval Blanc d'Henri IV est noire."